In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm
import time
from IPython.display import clear_output

import ee, geemap

In [2]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../../lib/')

In [3]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

# Part I: Get the locations from Google Drive

In [4]:
sheet_id = '13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos'
# sheet_id = '1XqOtPkiE_Q0dfGSoyxrH730RkwrTczcRbDeJJpqRByQ'
sheet_name = 'mozambique'
# sheet_name = 'sample_1'
drive_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
drive_url

'https://docs.google.com/spreadsheets/d/13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos/gviz/tq?tqx=out:csv&sheet=mozambique'

In [5]:
#Read in a google sheets with the ground truth data from string drive_loc
df = pd.read_csv(drive_url)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19)
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01)
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31)
5,moz-5,-25.353037,32.327312,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61)


In [6]:
#look for any NaN values in the column 'date' or 'Longitude' or 'Latitude' in df_merged and drop them
df = df.dropna(subset=['Date', 'Longitude', 'Latitude']).reset_index(drop=True)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19)
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01)
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31)
5,moz-5,-25.353037,32.327312,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61)


In [7]:
assert(len(df[df['Date'].isna() | df['Longitude'].isna() | df['Latitude'].isna()]) == 0)

# Part II: Select subset for training

Here we iterate over the df and iteratively pick and adjust the locations we want to keep

In [8]:
df.columns

Index(['ID', 'Latitude', 'Longitude', 'Class', 'Date', 'Site', 'Source',
       'Country', 'Notes'],
      dtype='object')

In [9]:
ee.Initialize()
# ee.Authenticate()

In [10]:
output = df.copy()
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19)
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01)
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31)
5,moz-5,-25.353037,32.327312,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61)


In [11]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

In [12]:
INDEX = 0
sampling_buffer_m = 5
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Manual Loop

Keep running the following 2 cells until you're done with the manual sampling.


In [13]:
s1_s2, sample = qm.get_s1s2_data(output, Map, INDEX, display_smap=True, mosaic_method='qm', sampling_buffer_m=sampling_buffer_m, median_samples=5,roi_buffer_m=400, obia=True)

Index:  0  ID:  moz-0 Class:  sand  Site:  fluvial
Search window from 2021-01-22 to 2022-01-22


Map(center=[-25.571042990148314, 32.252410081049256], controls=(ZoomControl(options=['position', 'zoom_in_text…

## Use this immediately after the map is refreshed to either select or discard the sample

INDEX will auto update on calling this function

In [14]:
output, INDEX = qm.get_training_sample(output, s1_s2, sample, Map, INDEX, display_clusters=False, obia=True)
output.iloc[INDEX-1]

New marker accepted
Kept Observation


ID                                                         moz-5
Latitude                                              -25.353037
Longitude                                              32.327312
Class                                                       sand
Date                                                  2022-07-22
Site                                                     fluvial
Source              https://portals.landfolio.com/mozambique/en/
Country                                               Mozambique
Notes                         License 8348C Applied (2016-09-61)
B2_mean                                                      NaN
B3_mean                                                      NaN
B4_mean                                                      NaN
B8_mean                                                      NaN
B8A_mean                                                     NaN
B11_mean                                                     NaN
B12_mean                 

## Final Output

In [21]:
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,B2_mean,...,B11_mean,B12_mean,VV_mean,VH_mean,mTGSI_mean,BSI_mean,NDWI_mean,keep,location_tweaked,class_code
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for,1024.304688,...,3794.05957,3441.761475,-13.875913,-24.152828,0.183746,0.244367,-0.246776,True,False,1
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
5,moz-5,-25.353037,32.327312,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99


In [120]:
output.to_csv('../../data/labels/mozambique-gt.csv')
#save fm_df to a pickle file
output.to_pickle('../../data/labels/mozambique-gt.pkl')

In [24]:
df = pd.read_csv('../../data/labels/mozambique-gt.csv')
df

,Unnamed: 0,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,...,B11_mean,B12_mean,VV_mean,VH_mean,mTGSI_mean,BSI_mean,NDWI_mean,keep,location_tweaked,class_code
0,0,moz-0,-25.569669,32.252534,sand,7/22/21,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for,...,4961.000000,4555.420410,-17.552206,-26.786810,0.173039,0.228689,-0.231081,True,True,1
1,1,moz-1,-24.505472,33.012067,sand,7/22/22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...,...,4737.139648,4739.217285,-16.774797,-27.074543,0.201724,0.234520,-0.251508,True,False,1
2,2,moz-2,-24.647689,33.283443,sand,8/15/19,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19),...,4133.804688,4107.097656,-16.950619,-29.517706,0.211118,0.245005,-0.296115,True,False,1
3,3,moz-3,-24.633682,33.283166,sand,8/15/19,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01),...,3819.920410,3836.006592,-17.742294,-31.180273,0.223607,0.255296,-0.298279,True,False,1
4,4,moz-4,-25.563075,32.209854,sand,7/22/21,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31),...,5094.271973,4696.789063,-15.571375,-25.351002,0.164142,0.216804,-0.240797,True,False,1
5,5,moz-5,-25.353174,32.327378,sand,7/22/22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61),...,4922.695313,4636.310547,-14.956991,-22.532507,0.171499,0.216569,-0.315728,True,True,1


In [ ]:
# write df to a new excel sheet in '../../data/labels/mozambique-gt.xlsx'